In [1]:
import datasets
import pickle as pkl
from tqdm.auto import tqdm

/home/haskari/miniconda3/envs/paraphrase/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def get_bad_indice(name,model):
    if model!='gpt3.5-T':
        if name=='cnn':
            with open('{}/data_paraphrase/cnn.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
        elif name=='xsum':
            with open('{}/data_paraphrase/xsum_capped_random.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
            
        elif name=='news':
            with open('{}/data_paraphrase/news_capped_random.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
        
        else:
            with open('{}/data_paraphrase/reddit_capped_random.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
    else:
        print('here')
        if name=='cnn':
            with open('{}/data_paraphrase/cnn.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
        elif name=='xsum':
            with open('{}/data_paraphrase/xsum.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
            
        elif name=='news':
            with open('{}/data_paraphrase/news.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
        
        else:
            with open('{}/data_paraphrase/reddit.pkl'.format(model), 'rb') as f:
                paraphrased_summaries=pkl.load(f)
        
    bad_index=[]
    for idx,sum in enumerate(paraphrased_summaries):
        if sum == []:
            bad_index.append(idx)
    return bad_index

In [5]:
#paraphrased mistral-7b

p_data_mistral_cnn = datasets.load_from_disk("llama2/saved_models/paraphrase-LlaMa2-cnn/")

p_data_mistral_xsum = datasets.load_from_disk("llama2/saved_models/paraphrase-LlaMa2-xsum/")

p_data_mistral_news = datasets.load_from_disk("llama2/saved_models/paraphrase-LlaMa2-news/")

p_data_mistral_reddit = datasets.load_from_disk("llama2/saved_models/paraphrase-LlaMa2-reddit/")


#original mistral-7b

data_mistral_cnn = datasets.load_from_disk("llama2/saved_models/original-LlaMa2-cnn/")

bad_index=get_bad_indice('cnn','llama2')
data_mistral_cnn=data_mistral_cnn.select(i for i in range(len(data_mistral_cnn)) 
                 if i not in set(bad_index))


data_mistral_xsum = datasets.load_from_disk("llama2/saved_models/original-LlaMa2-xsum/")

bad_index=get_bad_indice('xsum','llama2')
data_mistral_xsum=data_mistral_xsum.select(i for i in range(len(data_mistral_xsum)) 
                 if i not in set(bad_index))

data_mistral_news = datasets.load_from_disk("llama2/saved_models/original-LlaMa2-news/")

bad_index=get_bad_indice('news','llama2')
data_mistral_news=data_mistral_news.select(i for i in range(len(data_mistral_news)) 
                 if i not in set(bad_index))


data_mistral_reddit = datasets.load_from_disk("llama2/saved_models/original-LlaMa2-reddit/")

bad_index=get_bad_indice('reddit','llama2')
data_mistral_reddit=data_mistral_reddit.select(i for i in range(len(data_mistral_reddit)) 
                 if i not in set(bad_index))

Parameter 'indices'=<generator object <genexpr> at 0x7fc8c2506ce0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [6]:
p_data_mistral_reddit

Dataset({
    features: ['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title', 'original_article', 'article', 'highlights', 'segment_idxs', 'mapping', 'article_length', 'highlights_length', 'model_summaries', 'mapping_gen'],
    num_rows: 323
})

In [7]:
data_mistral_reddit

Dataset({
    features: ['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title', 'original_article', 'article', 'highlights', 'segment_idxs', 'mapping', 'article_length', 'highlights_length', 'model_summaries', 'mapping_gen'],
    num_rows: 339
})

In [8]:
p_data_mistral_reddit['highlights'][3]

['wore a hot dog costume to school and while i ran during cross country practice and got called wiener by everyone for the years to come.']

In [9]:
p_data_mistral_reddit['model_summaries'][3]

['The author was nicknamed "Wiener" in middle school due to wearing a hot dog costume on Halloween and being teased by students and a teacher, who found it hilarious but the author now realizes was kind of fucked up']

In [10]:
#from MENLI import MENLI
from MENLI.MENLI import MENLI

scorer = MENLI(direction="avg", formula="e", src=False, nli_weight=0.3, combine_with="BERTScore-F", model="D")
res=[]
count=0
# refs and hyps in form of list of String
for cand, ref in tqdm(zip(p_data_mistral_cnn['highlights'],p_data_mistral_cnn['model_summaries'])):
    count+=1
    # if count>=20:
    #     break
    if len(cand)!=len(ref):
        continue
    if not cand or not ref:
        continue
    scores=scorer.score_all(srcs=[],refs=cand, hyps=ref) 

    res.append(sum(scores)/len(scores))
    
   
return sum(res)/len(res)

0it [00:00, ?it/s]

computing metric scores....
computing nli scores (direction avg + formula e)...


1it [00:01,  1.92s/it]

averaging....
combing scores...
computing metric scores....


/home/haskari/miniconda3/envs/paraphrase/lib/python3.12/site-packages/bert_score/scorer.py:178: UserWarning: Overwriting the previous importance weights.
  warnings.warn("Overwriting the previous importance weights.")
3it [00:02,  1.20it/s]

...

16it [00:08,  1.90it/s]

computing nli scores (direction avg + formula e)...
averaging....
combing scores...
computing metric scores....


17it [00:09,  1.64it/s]

computing nli scores (direction avg + formula e)...
averaging....
combing scores...


19it [00:09,  2.03it/s]


In [11]:
len(res)

9

In [12]:
res

[0.4038774366533841,
 0.40349568840067307,
 0.4383746677079836,
 0.3615079469181343,
 0.42108726991601736,
 0.5130963350462044,
 0.41315686720297434,
 0.46100161126459893,
 0.4171322914520195]

In [13]:
sum(res)/len(res)

0.4258589016179989